Reading in the DF

In [ ]:
import pandas as pd
import requests

url = 'https://docs.google.com/spreadsheets/d/1AWHOzyVWws-VSe5JsMb4gmJHHB5vCMy0OXbPH4pPjwA/edit?usp=sharing'

r = requests.get(f'https://docs.google.com/spreadsheet/ccc?key=' + url.split('/')[-2] + '&output=csv')
open('dataset.csv', 'wb').write(r.content)
df = pd.read_csv('dataset.csv')

In [ ]:
import numpy as np

# df['C5_PICKS'] = df['C5_PICKS'].astype('int', errors="ignore")
# df['C6_PICKS'] = df['C6_PICKS'].astype('int', errors="ignore")
df["guess_order"] = np.nan
df['guess_order'] = df['guess_order'].astype('object')

df["as_C1_guess"] = np.nan
df["as_C1_guess_correct"] = 0

df["as_C2_guess"] = np.nan
df["as_C2_guess_correct"] = 0

df["as_C3_guess"] = np.nan
df["as_C3_guess_correct"] = 0

df["as_C4_guess"] = np.nan
df["as_C4_guess_correct"] = 0

df["as_C5_guess"] = np.nan
df["as_C5_guess_correct"] = 0

df["as_C6_guess"] = np.nan
df["as_C6_guess_correct"] = 0


df.head()

,GAME,ROUND,PLAYERS,DESCRIPTION,TARGET,C1_CARD,C1_PICKS,C2_CARD,C2_PICKS,C3_CARD,...,as_C2_guess,as_C2_guess_correct,as_C3_guess,as_C3_guess_correct,as_C4_guess,as_C4_guess_correct,as_C5_guess,as_C5_guess_correct,as_C6_guess,as_C6_guess_correct
0,13,4,4,And next we'll see talking dogs and a dodo bird,63,63,0,32,1,45,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1,13,11,4,to be love,36,41,0,22,0,40,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
2,24,3,4,plutonium lvl20,39,2,1,39,1,23,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
3,24,4,4,I think I can I think I can!,12,5,0,12,3,40,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
4,24,6,4,child lost in space,47,47,3,17,0,53,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0


Initializing Clip

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
import datasets
import numpy as np
import pandas as pd
import torch
from PIL import Image
import requests

from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch32"

# we initialize a tokenizer, image processor, and the model itself
tokenizer = CLIPTokenizerFast.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id).to(device)

CSV Reading

In [ ]:
df.at[501, "DESCRIPTION"] = "scrubs reference" # IDK but model breaks, maybe description is too long
df.at[569, "DESCRIPTION"] = "henry V" # same

In [ ]:
def getGuess(row):
    numPlayers = row["PLAYERS"]
    cards = ["C1_CARD", "C2_CARD", "C3_CARD", "C4_CARD"]
    if numPlayers > 4:
        cards.append("C5_CARD")
    if numPlayers > 5:
        cards.append("C6_CARD")

    card_names = [str(int(card)) + ".jpg" for card in row[cards]]
    #print("cards", card_names)
    descrip = row["DESCRIPTION"]
    #print(descrip, descrip)
    text_prompts=[descrip, " "]
    images=[Image.open(r'/content/' + i)  for i in card_names]
    # if only 1 prompt, then it matches all of them to it
    inputs = inputs = processor(text=text_prompts, images=images, return_tensors="pt", padding=True)

    inputs= inputs.to(device)

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

    res = pd.DataFrame(probs.cpu().detach().numpy()*100, columns=text_prompts, index=list(card_names))
    res = res[descrip].sort_values().index.str[:-4].tolist()
    res.reverse()
    return res

In [ ]:
def fillTable(index, row, results):
    df.at[index, "guess_order"] = [eval(i) for i in results]
    numPlayers = row["PLAYERS"]

    players = ["C1", "C2", "C3", "C4", "C5", "C6"]

    for player in players[:numPlayers]:
        playedCard = row[player + "_CARD"]

        if playedCard == row["TARGET"]:
            continue

        guess = results[0]

        if playedCard == int(guess):
            guess = results[1]

        df.at[index, "as_" + player + "_guess"] = guess
        if int(guess) == row["TARGET"]:
            df.at[index, "as_" + player + "_guess_correct"] = 1


In [ ]:
from google.colab import files

def downloadDF(index):
    pdfName = 'df' + str(index) + '.csv'
    df.to_csv(pdfName)
    files.download(pdfName)

In [ ]:
for index, row in df.iterrows():
    # if index >= 0 and index < 100: # 5000 and index < 20000:
    try:
        results = getGuess(row)
        fillTable(index, row, results)
    except:
        print("except at index", index)

downloadDF(index)

except at index 250


Token indices sequence length is longer than the specified maximum sequence length for this model (93 > 77). Running this sequence through the model will result in indexing errors


except at index 2797
except at index 4172
except at index 4286


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
downloadDF(2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
"""
for index, row in df.iterrows():
    if index >= 0 # 5000 and index < 20000:
        try:
            numPlayers = row["PLAYERS"]
            cards = ["C1_CARD", "C2_CARD", "C3_CARD", "C4_CARD"]
            if numPlayers > 4:
                cards.append("C5_CARD")
            if numPlayers > 5:
                cards.append("C6_CARD")

            card_names = [str(int(card)) + ".jpg" for card in row[cards]]
            #print("cards", card_names)
            descrip = row["DESCRIPTION"]
            #print(descrip, descrip)
            text_prompts=[descrip, " "]
            images=[Image.open(r'/content/' + i)  for i in card_names]
            # if only 1 prompt, then it matches all of them to it
            inputs = inputs = processor(text=text_prompts, images=images, return_tensors="pt", padding=True)

            inputs= inputs.to(device)

            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)

            res = pd.DataFrame(probs.cpu().detach().numpy()*100, columns=text_prompts, index=list(card_names))
            #print(res)

            guess = res[descrip].idxmax()[:-4]
            df.at[index, "bot_guess"] = guess
            if int(guess) == row["TARGET"]:
                df.at[index, "bot_correct"] = 1
        except:
            print("except at index", index)
"""

excepts at:

except at index 2797

except at index 4172

except at index 4286

except at index 16148

In [ ]:
df

,GAME,ROUND,PLAYERS,DESCRIPTION,TARGET,C1_CARD,C1_PICKS,C2_CARD,C2_PICKS,C3_CARD,...,C4_CARD,C4_PICKS,C5_CARD,C5_PICKS,C6_CARD,C6_PICKS,correct_guesses,percent_correct,bot_guess,bot_correct
0,13,4,4,And next we'll see talking dogs and a dodo bird,63,63,0,32,1,45,...,10,0,NaN,NaN,NaN,NaN,0,0.000000,NaN,0
1,13,11,4,to be love,36,41,0,22,0,40,...,36,2,NaN,NaN,NaN,NaN,2,0.666667,NaN,0
2,24,3,4,plutonium lvl20,39,2,1,39,1,23,...,27,1,NaN,NaN,NaN,NaN,1,0.333333,NaN,0
3,24,4,4,I think I can I think I can!,12,5,0,12,3,40,...,42,0,NaN,NaN,NaN,NaN,3,1.000000,NaN,0
4,24,6,4,child lost in space,47,47,3,17,0,53,...,41,0,NaN,NaN,NaN,NaN,3,1.000000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23303,75842,1,4,broken,3,67,0,3,3,40,...,11,0,NaN,NaN,NaN,NaN,3,1.000000,NaN,0
23304,75842,3,4,nurture,41,25,1,26,1,62,...,41,1,NaN,NaN,NaN,NaN,1,0.333333,NaN,0
23305,75842,4,4,Together,13,13,1,83,1,31,...,36,1,NaN,NaN,NaN,NaN,1,0.333333,NaN,0
23306,75842,10,4,Wow,27,27,2,12,0,39,...,70,1,NaN,NaN,NaN,NaN,2,0.666667,NaN,0
